NOTAM_to_Spatial
or in fact, NOTAM to spatio-temporal

In [1]:
import pandas as pd
import re
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

In [2]:
# function to extract coordinates and radius (if present)
#def get_coordinates(name):
path=r"D:\UNIGIS\MASTER\CreatedData\TFR_Project\ZSE_2021_All_Aug-Oct_revisited.xlsx"
df = pd.read_excel(path)

In [3]:
df

,Source,Location,NOTAM #,Class,Issue Date (UTC),Effective Date (UTC),Cancel Date (UTC),Expiration Date (UTC),NOTAM Condition or LTA Subject
0,ZSE_2021-08-01,ZSE,1/3680,Airspace,07/22/2021 1205,07/22/2021 1400,08/19/2021 0523,09/22/2021 0500,!FDC 1/3680 ZSE MN..AIRSPACE 7NM N THOMPSON FA...
1,ZSE_2021-08-01,ZSE,1/6739,Airspace,07/10/2021 0110,07/10/2021 1400,08/17/2021 0450,09/10/2021 0500,"!FDC 1/6739 ZSE MT..AIRSPACE 9NM S OF TROY, MT..."
2,ZSE_2021-08-01,ZSE,1/3581,Airspace,07/22/2021 0057,07/22/2021 1500,08/16/2021 0147,09/22/2021 0330,!FDC 1/3581 ZSE ID..AIRSPACE 20NM SE OF LEWIST...
3,ZSE_2021-08-01,ZSE,1/7262,Airspace,07/12/2021 0114,07/12/2021 1300,08/15/2021 1012,09/12/2021 0500,!FDC 1/7262 ZSE ID..AIRSPACE 5NM NE OF KELLOGG...
4,ZSE_2021-08-01,ZSE,1/2358,Airspace,07/20/2021 0513,07/20/2021 1400,08/12/2021 0510,09/20/2021 0500,"!FDC 1/2358 ZSE MT..AIRSPACE 5NM N OF TROY, MT..."
...,...,...,...,...,...,...,...,...,...
253,ZSE_2021-10-04,ZSE,1/7554,Airspace,10/04/2021 1448,10/04/2021 1500,10/15/2021 1441,10/18/2021 0500,!FDC 1/7554 ZSE OR..AIRSPACE 24NM E OF ROSEBUR...
254,ZSE_2021-10-07,ZSE,1/1515,Airspace,10/07/2021 2152,10/07/2021 2230,10/08/2021 2059,10/14/2021 1423,!FDC 1/1515 ZSE OR..AIRSPACE 27NM W OF SUNRIVE...
255,ZSE_2021-10-17,ZSE,1/4201,Airspace,10/17/2021 1216,10/17/2021 1500,10/20/2021 0202,11/17/2021 0300,!FDC 1/4201 ZSE CA..AIRSPACE 35NM EAST OF CRES...
256,ZSE_2021-10-17,ZSE,1/4257,Airspace,10/17/2021 1527,10/17/2021 1530,10/17/2021 2107,10/18/2021 0500,"!FDC 1/4257 ZSE OR..AIRSPACE AURORA, OR..TEMPO..."


In [4]:
def get_coordinates(row):
#     # find all substrings with 6 digits before N and 7 digits plus W = 8
    sub_coords = "\w{6}N\w{8}"
    coordinates = re.findall(sub_coords,row["NOTAM Condition or LTA Subject"])
    coords_list = []
    for coord in coordinates:
        #replaces old coordinates = row["NOTAM Condition or LTA Subject"].findall(sub_coords)
        #north
        deg_lat = coord[:2]
        min_lat = coord[2:4]
        sec_lat = coord[4:6]
        dd_lat = float(deg_lat) + float(min_lat)/60 + float(sec_lat)/(60*60)
        
        #west
        deg_lon = coord[7:10]
        min_lon = coord[10:12]
        sec_lon = coord[12:14]
        dd_lon = -1*(float(deg_lon) + float(min_lon)/60 + float(sec_lon)/(60*60))
        
        coords_list.append((dd_lon,dd_lat))
        
    return coords_list
        
df["Coordinates"]=df.apply(get_coordinates,axis=1)

In [5]:
df["Coordinates"]

0             [(-115.26666666666667, 47.68333333333333)]
1                                   [(-115.9675, 48.34)]
2      [(-116.82194444444444, 46.18055555555555), (-1...
3      [(-116.21361111111112, 47.68722222222222), (-1...
4      [(-116.03888888888889, 48.638888888888886), (-...
                             ...                        
253    [(-122.58944444444444, 43.63333333333333), (-1...
254          [(-121.75416666666666, 43.916666666666664)]
255    [(-123.38333333333334, 41.78333333333333), (-1...
256                      [(-120.65, 43.800555555555555)]
257                                 [(-120.85, 45.7625)]
Name: Coordinates, Length: 258, dtype: object

In [6]:
def get_geometry(row):
    #transform float coordinates from get_coordinates(row)
    coords = row["Coordinates"]
    #find points
    if len(coords)==1:
        geom = Point(coords[0][0],coords[0][1])
    elif len(coords)==2:
        geom = LineString(coords)
    elif len(coords)>2:
        geom = Polygon(coords) 
    else:
        geom = None
        
    return geom
df["geometry"] = df.apply(get_geometry,axis=1)

D:\anaconda3\envs\master_env\lib\site-packages\pandas\core\dtypes\cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [7]:
df["geometry"]

0          POINT (-115.26666666666667 47.68333333333333)
1                                POINT (-115.9675 48.34)
2      POLYGON ((-116.82194444444444 46.1805555555555...
3      POLYGON ((-116.21361111111112 47.6872222222222...
4      POLYGON ((-116.03888888888889 48.6388888888888...
                             ...                        
253    POLYGON ((-122.58944444444444 43.6333333333333...
254       POINT (-121.75416666666666 43.916666666666664)
255    POLYGON ((-123.38333333333334 41.7833333333333...
256                   POINT (-120.65 43.800555555555555)
257                              POINT (-120.85 45.7625)
Name: geometry, Length: 258, dtype: object

In [8]:
def get_radius(row):
    #find radius
    sub_pt_radius = "\w{1,}\\.\w{1,}NM RADIUS" ## if fraction like 1.5NM RADIUS is given
    sub_radius = "\w{1,}NM\sRADIUS"
    sub_wr_radius = "\w{1,}NM\nRADIUS"
    sub_spNM_radius = "\w{1,} NM RADIUS"
    
    #check for decimal fraction, needs to be done first, otherwise 1.5NM would result in 5NM
    radius = re.findall(sub_pt_radius,row["NOTAM Condition or LTA Subject"])
    
    #when there is no decimal fraction, look for most common case
    if len(radius)==0:
        radius = re.findall(sub_radius,row["NOTAM Condition or LTA Subject"])
    else:
        radius = radius     
    
    #when there is a line wrap btn NM and radius, its len is still 0 instead of 1:
    if len(radius)==0:
        radius = re.findall(sub_wr_radius,row["NOTAM Condition or LTA Subject"])    
    else:
        radius = radius
    
    #when there is a space btn number and NM, its len is still 0 instead of 1:
    if len(radius)==0:
        radius = re.findall(sub_spNM_radius,row["NOTAM Condition or LTA Subject"])      
    else:
        radius = radius
        
    #so far it is 5NM RADIUS or 12NM RADIUS or 2 NM RADIUS or 1.5NM RADIUS, so cut down to the numbers
    for chars in radius:
        #one digit
        if len(chars)==10:
            radius = chars[0]
        #two digits
        elif len(chars)==11: 
            radius = chars[:2]
        #fraction
        elif len(chars)==12: 
            radius = chars[:3]
        #fraction and tens (not known if any)
        elif len(chars)==13: 
            radius = chars[:4]
        #everything else is erroneous
        else:
            radius = "WrongLength"
        
    return radius
df["Radius"]=df.apply(get_radius,axis=1)

In [9]:
print(df["Radius"])

0       6
1       5
2      []
3      []
4      []
       ..
253    []
254     5
255    []
256     4
257     3
Name: Radius, Length: 258, dtype: object


In [10]:
#with Radius being a list that causes issues during JSON export,it needs to get changed:
df["Radius"] =df["Radius"].astype('string')

In [11]:
#single numbers are needed as radius instead of list residuals
def convert_radius(row):
    radius = row["Radius"]
    if radius == "[]":
        radius = 0
    return radius
df["Radius"]=df.apply(convert_radius,axis=1)

In [12]:
#string has still [] as values which cannot be converted to number format, so

df["Radius"] =df["Radius"].astype('float')

In [13]:
#turn radius from NM to m for buffer
def radius_to_m(row):
    radius_m = row["Radius"]*1852
    return radius_m
df["Radius_m"]=df.apply(radius_to_m,axis=1)

In [14]:
#look at ALL rows
pd.set_option('display.max_rows', None)
print(df["Radius_m"])
#print(df.loc[174:233,"Radius"])

0      11112.0
1       9260.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7      11112.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14     12964.0
15      9260.0
16      9260.0
17      9260.0
18         0.0
19      9260.0
20      9260.0
21         0.0
22     22224.0
23      9260.0
24      9260.0
25      5556.0
26      9260.0
27      5556.0
28         0.0
29      9260.0
30      9260.0
31     12964.0
32      9260.0
33         0.0
34         0.0
35      9260.0
36      9260.0
37      9260.0
38         0.0
39         0.0
40      9260.0
41         0.0
42     11112.0
43         0.0
44         0.0
45      9260.0
46      9260.0
47         0.0
48         0.0
49         0.0
50     12964.0
51     12964.0
52     11112.0
53      3704.0
54      9260.0
55     12964.0
56         0.0
57      9260.0
58         0.0
59         0.0
60         0.0
61      9260.0
62         0.0
63      7408.0
64         0.0
65         0.0
66        

In [15]:
#convert the date columns into date format
#df["Issue Date (UTC)"] = pd.to_datetime(df["Issue Date (UTC)"], errors='coerce')
# --> still text in GIS

In [16]:
df["Issue Date (UTC)"]

0      07/22/2021 1205
1      07/10/2021 0110
2      07/22/2021 0057
3      07/12/2021 0114
4      07/20/2021 0513
5      07/20/2021 0521
6      07/07/2021 2349
7      07/29/2021 1308
8      08/01/2021 0424
9      07/19/2021 0414
10     07/27/2021 0459
11     08/01/2021 0318
12     07/26/2021 0401
13     07/31/2021 2322
14     07/30/2021 0126
15     07/09/2021 0451
16     07/19/2021 0215
17     07/29/2021 1338
18     07/30/2021 1942
19     07/25/2021 0420
20     08/01/2021 1620
21     07/27/2021 0341
22     08/01/2021 0100
23     07/31/2021 1422
24     07/21/2021 1333
25     08/01/2021 1555
26     08/01/2021 0410
27     07/30/2021 2027
28     07/27/2021 0505
29     07/31/2021 1954
30     07/30/2021 0248
31     08/02/2021 0506
32     08/02/2021 0414
33     08/02/2021 0419
34     08/02/2021 0417
35     08/02/2021 1509
36     08/02/2021 1612
37     08/02/2021 0232
38     08/02/2021 0206
39     08/03/2021 0424
40     08/03/2021 1727
41     08/03/2021 0513
42     08/03/2021 1726
43     08/0

In [17]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry=df["geometry"])

In [18]:
print(gdf)

             Source Location NOTAM #     Class Issue Date (UTC)  \
0    ZSE_2021-08-01      ZSE  1/3680  Airspace  07/22/2021 1205   
1    ZSE_2021-08-01      ZSE  1/6739  Airspace  07/10/2021 0110   
2    ZSE_2021-08-01      ZSE  1/3581  Airspace  07/22/2021 0057   
3    ZSE_2021-08-01      ZSE  1/7262  Airspace  07/12/2021 0114   
4    ZSE_2021-08-01      ZSE  1/2358  Airspace  07/20/2021 0513   
5    ZSE_2021-08-01      ZSE  1/2359  Airspace  07/20/2021 0521   
6    ZSE_2021-08-01      ZSE  1/5565  Airspace  07/07/2021 2349   
7    ZSE_2021-08-01      ZSE  1/7785  Airspace  07/29/2021 1308   
8    ZSE_2021-08-01      ZSE  1/9079  Airspace  08/01/2021 0424   
9    ZSE_2021-08-01      ZSE  1/1751  Airspace  07/19/2021 0414   
10   ZSE_2021-08-01      ZSE  1/5613  Airspace  07/27/2021 0459   
11   ZSE_2021-08-01      ZSE  1/9074  Airspace  08/01/2021 0318   
12   ZSE_2021-08-01      ZSE  1/4854  Airspace  07/26/2021 0401   
13   ZSE_2021-08-01      ZSE  1/9018  Airspace  07/31/2021 232

In [19]:
#are there columns making trouble? Use only these:
gdf= gdf[["Source","Location", "NOTAM #", "Issue Date (UTC)", "Effective Date (UTC)", "Cancel Date (UTC)", "Expiration Date (UTC)", "NOTAM Condition or LTA Subject","Radius","Radius_m","geometry"]]

In [20]:
#get rid of all rows w/o geometry (not necessary for ArcGIS Pro JSON to Features but for buffer)
gdf = gdf[gdf["geometry"]!=None]

In [21]:
#gdf.geom_type

In [22]:
#prepare buffer
gdf_buffered = gdf.copy()
gdf_buffered = gdf_buffered.to_crs("EPSG:2163")

In [23]:
#do buffer by radius: SHOULD be 0 for polygons, do it for all like this is FAST
gdf_buffered["geometry"] = gdf_buffered.buffer(gdf["Radius_m"], resolution=16)

In [24]:
#buffer by radius attempt per row, SLOW
# def get_buffer(row):
#     geom = row["geometry"]
#     for sort in gdf.geom_type:
#         if sort == Point:
#     #if row["geometry"].geom_type == 0:
#         geom = geom.buffer(row["Radius_m"], resolution=16)
#     return geom
# gdf["geometry"]=df.apply(get_buffer,axis=1)

In [25]:
gdf["geometry"]

0                            POINT (-115.26667 47.68333)
1                            POINT (-115.96750 48.34000)
2      POLYGON ((-116.82194 46.18056, -116.73333 46.1...
3      POLYGON ((-116.21361 47.68722, -115.82139 47.7...
4      POLYGON ((-116.03889 48.63889, -115.79444 48.6...
5      POLYGON ((-117.70278 46.32861, -117.23306 46.3...
6      POLYGON ((-115.67111 45.71639, -115.38389 45.7...
7                            POINT (-122.36611 49.02083)
8      POLYGON ((-119.95722 48.90083, -119.91972 48.4...
9      POLYGON ((-116.06000 47.03333, -115.89611 47.0...
10     POLYGON ((-121.20528 42.97472, -120.69000 42.9...
11     POLYGON ((-117.03167 48.29417, -116.87028 48.2...
12     POLYGON ((-118.64444 48.50000, -118.29444 48.5...
13     POLYGON ((-122.45222 44.04139, -122.26417 44.0...
14                           POINT (-117.60000 45.86667)
15                           POINT (-116.58333 46.93333)
16                           POINT (-116.17528 46.83972)
17                           PO

In [26]:
#gdf_buffered_clean.head()

In [27]:
#gdf_buffered_clean = gdf_buffered_clean[gdf_buffered_clean["geometry"]!=None]

In [28]:
#create geojson of gdf, points and polygons
gdf.to_file(filename='gdf_first.geojson', driver='GeoJSON')

D:\anaconda3\envs\master_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [29]:
#create geojson of gdf_buffered, just polygons
#back to WGS84 for geojson creation
gdf_buffered = gdf_buffered.to_crs("EPSG:4326")
gdf_buffered.to_file(filename='gdf_buffered.geojson', driver='GeoJSON')

D:\anaconda3\envs\master_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [30]:
#shape file of course only works with uniform geometries
#gdf_buffered.to_file('gdf_buffered.shp', driver='ESRI Shapefile')

In [31]:
#plot result into new window
%matplotlib qt
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.to_crs("EPSG:4326")
ax = world[world.continent == 'North America'].plot(
    color='white', edgecolor='black')
gdf_buffered.plot(ax=ax, color='red')

<AxesSubplot:>

In [32]:
gdf_buffered["geometry"].head()

0    POLYGON ((-115.12036 47.70223, -115.11826 47.6...
1    POLYGON ((-115.84424 48.35653, -115.84234 48.3...
2    POLYGON ((-116.82194 46.18056, -116.73333 46.1...
3    POLYGON ((-116.21361 47.68722, -115.82139 47.7...
4    POLYGON ((-116.03889 48.63889, -115.79444 48.6...
Name: geometry, dtype: geometry